In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import re
import seaborn as sns
sns.set_theme(style="whitegrid")

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.efficientnet import EfficientNetB4
from tensorflow.keras.models import save_model

In [5]:
physical_devices = tf.config.list_physical_devices("GPU")
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
features = np.load("features.npy")
labels = np.load("labels.npy")

In [7]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [8]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2)

In [9]:
train_features.shape

(43444, 128, 128, 3)

In [10]:
train_labels.shape

(43444, 2)

In [11]:
test_features.shape

(10861, 128, 128, 3)

In [12]:
test_labels.shape

(10861, 2)

In [14]:
input_shape=(128,128,3)
classes = 2
epoch = 10
batch_size = 64

efficientnet_b4 = EfficientNetB4(weights=None, input_shape=input_shape, classes=classes)

# efficientnet_b4.summary()

efficientnet_b4.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

early_stop= EarlyStopping(monitor='val_loss',patience=10)

mcp_save = ModelCheckpoint('mdl_efficientnet_b4_1.hdf5', save_best_only=True, monitor='val_loss', mode='min')

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

efficientnet_b4.fit(train_features, train_labels,
          epochs=epoch,
          batch_size=batch_size,
          validation_data=(test_features,test_labels), 
          callbacks=[early_stop, learning_rate_reduction])


Epoch 1/10
679/679 [==============================] - 352s 496ms/step - loss: 1.6859 - accuracy: 0.7340 - val_loss: 0.3977 - val_accuracy: 0.8099
Epoch 2/10
679/679 [==============================] - 331s 487ms/step - loss: 0.4289 - accuracy: 0.8644 - val_loss: 0.2459 - val_accuracy: 0.9035
Epoch 3/10
679/679 [==============================] - 329s 485ms/step - loss: 0.3668 - accuracy: 0.8879 - val_loss: 2.3088 - val_accuracy: 0.6690
Epoch 4/10
679/679 [==============================] - 330s 485ms/step - loss: 0.3788 - accuracy: 0.8697 - val_loss: 0.2831 - val_accuracy: 0.8974
Epoch 5/10
679/679 [==============================] - 330s 487ms/step - loss: 0.2904 - accuracy: 0.9102 - val_loss: 0.2177 - val_accuracy: 0.9385
Epoch 6/10
679/679 [==============================] - 330s 486ms/step - loss: 0.1999 - accuracy: 0.9340 - val_loss: 0.2960 - val_accuracy: 0.8540
Epoch 7/10
679/679 [==============================] - 330s 486ms/step - loss: 0.3182 - accuracy: 0.8985 - val_loss: 0.1590 -

In [15]:
filepath = '../models/efficientnet_b4'
save_model(efficientnet_b4, filepath)

metrics=pd.DataFrame(efficientnet_b4.history.history)
metrics

INFO:tensorflow:Assets written to: ../models/efficientnet_b4/assets


,loss,accuracy,val_loss,val_accuracy,lr
0,0.987557,0.786829,0.397720,0.809870,0.001
1,0.408515,0.868635,0.245944,0.903508,0.001
2,0.366651,0.889076,2.308829,0.668999,0.001
3,0.342333,0.879983,0.283053,0.897431,0.001
4,0.282706,0.914211,0.217715,0.938496,0.001
5,0.218448,0.925790,0.296003,0.853973,0.001
6,0.242423,0.916053,0.158989,0.940153,0.001
7,0.093973,0.964897,0.070306,0.974772,0.001
8,0.073174,0.974059,0.060619,0.975785,0.001
9,0.049683,0.982368,0.041687,0.985176,0.001


In [16]:
results = efficientnet_b4.evaluate(test_features,test_labels)
print(f"test acc = {results[1]}\ntest loss = {results[0]}")

340/340 [==============================] - 21s 59ms/step - loss: 0.0417 - accuracy: 0.9852
test acc = 0.9851763248443604
test loss = 0.04168718680739403
